In [1]:
from datetime import datetime

from pyspark import Row
from pyspark.sql import SparkSession, Window, functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, TimestampType, IntegerType

In [2]:
spark = (SparkSession
         .builder
         .master("local[*]")
         .appName("data_exploration")
         .config("spark.driver.memory", "2g")
         .getOrCreate())

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/16 19:52:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc.uiWebUrl

'http://192.168.0.16:4040'

On visual inspection, each file has several lines of headers that we need to strip (starting with "#").

In [4]:
parsed_input_rdd = (sc
                   .textFile("../data/kis_tot_2*", 2)
                   .map(lambda x: x.split(",")[0])
                   .filter(lambda x: not x.startswith("#")))

In [5]:
parsed_input_rdd.take(5)

['2003-04-01 00:10:00  235_T_obs           De Kooy waarneemterrein                         52.926944           4.781111            1.2                 1                   3.1                 1.4                 2.2                                                         3.1                 2.716426            3.2                 94                                      ',
 '2003-04-01 00:20:00  235_T_obs           De Kooy waarneemterrein                         52.926944           4.781111            1.2                 1                   3.3                 1.7                 2.4                                                         3.1                 2.913063            3.3                 94                                      ',
 '2003-04-01 00:30:00  235_T_obs           De Kooy waarneemterrein                         52.926944           4.781111            1.2                 1                   3.4                 1.9                 2.5                                      

The file appears to be in a fixed width format. The columns can be parsed by extracting partial strings using the column width given by a text editor. This is brittle and will only work if all files have the same column widths.

The fields in the data that we need to parse are:

| Field                | Description                                                         |
|----------------------|---------------------------------------------------------------------|
| `DTG`                | date of measurement                                                 |
| `LOCATION`           | location of the meteorological station                              |
| `NAME`               | name of the meteorological station                                  |
| `LATITUDE`           | in degrees (WGS84)                                                  |
| `LONGITUDE`          | in degrees (WGS84)                                                  |
| `ALTITUDE`           | in 0.1 m relative to Mean Sea Level (MSL)                           |
| `U_BOOL_10`          | air humidity code boolean 10' unit                                  |
| `T_DRYB_10`          | air temperature 10' unit Celcius degrees                            |
| `TN_10CM_PAST_6H_10` | air temperature minimum 0.1m 10' unit Celcius degrees               |
| `T_DEWP_10`          | air temperature derived dewpoint - 10' unit Celcius degrees         |
| `T_DEWP_SEA_10`      | air temperature derived dewpoint- sea 10' unit Celcius degrees      |
| `T_DRYB_SEA_10`      | air temperature height oil platform 10 minutes unit Celcius degrees |
| `TN_DRYB_10`         | air temperature minimum 10' unit Celcius degrees                    |
| `T_WETB_10`          | air temperature derived wet bulb- 10' unit Celcius degrees          |
| `TX_DRYB_10`         | air temperature maximum 10' unit Celcius degrees                    |
| `U_10`               | relative air humidity 10' unit %                                    |
| `U_SEA_10`           | is relative sea air humidity 10' unit %                             |

In [6]:
def parse_float(string_to_parse: str):
    string = string_to_parse.rstrip()
    if string == "":
        return None
    else:
        return float(string_to_parse)


def parse_datetime(string_to_parse: str):
    string = string_to_parse.rstrip()
    return datetime.strptime(string, "%Y-%m-%d %H:%M:%S")


def parse_row(row: str):
    return Row(
        dtg=parse_datetime(row[:21]),
        location=row[21:41].rstrip(),
        name=row[41:89].rstrip(),
        latitude=parse_float(row[89:109]),
        longitude=parse_float(row[109:129]),
        altitude=parse_float(row[129:149]),
        u_bool_10=parse_float(row[149:169]),
        t_dryb_10=parse_float(row[169:189]),
        tn_10cm_past_6h_10=parse_float(row[189:209]),
        t_dewp_10=parse_float(row[209:229]),
        t_dewp_sea_10=parse_float(row[229:249]),
        t_dryb_sea_10=parse_float(row[249:269]),
        tn_dryb_10=parse_float(row[269:289]),
        t_wetb_10=parse_float(row[289:309]),
        tx_dryb_10=parse_float(row[309:329]),
        u_10=parse_float(row[329:349]),
        u_sea_10=parse_float(row[349:]),
    )

parsed_input_rdd = parsed_input_rdd.map(lambda x: parse_row(x))

In [7]:
parsed_input_rdd.take(5)

[Row(dtg=datetime.datetime(2003, 4, 1, 0, 10), location='235_T_obs', name='De Kooy waarneemterrein', latitude=52.926944, longitude=4.781111, altitude=1.2, u_bool_10=1.0, t_dryb_10=3.1, tn_10cm_past_6h_10=1.4, t_dewp_10=2.2, t_dewp_sea_10=None, t_dryb_sea_10=None, tn_dryb_10=3.1, t_wetb_10=2.716426, tx_dryb_10=3.2, u_10=94.0, u_sea_10=None),
 Row(dtg=datetime.datetime(2003, 4, 1, 0, 20), location='235_T_obs', name='De Kooy waarneemterrein', latitude=52.926944, longitude=4.781111, altitude=1.2, u_bool_10=1.0, t_dryb_10=3.3, tn_10cm_past_6h_10=1.7, t_dewp_10=2.4, t_dewp_sea_10=None, t_dryb_sea_10=None, tn_dryb_10=3.1, t_wetb_10=2.913063, tx_dryb_10=3.3, u_10=94.0, u_sea_10=None),
 Row(dtg=datetime.datetime(2003, 4, 1, 0, 30), location='235_T_obs', name='De Kooy waarneemterrein', latitude=52.926944, longitude=4.781111, altitude=1.2, u_bool_10=1.0, t_dryb_10=3.4, tn_10cm_past_6h_10=1.9, t_dewp_10=2.5, t_dewp_sea_10=None, t_dryb_sea_10=None, tn_dryb_10=3.2, t_wetb_10=3.011368, tx_dryb_10=3.4

As we are only interested in data for the weather station `De Bilt` indicated by the Location `260_T_a`, we filter other records out.

In [8]:
parsed_input_rdd = parsed_input_rdd.filter(lambda row: row.location == "260_T_a")

In [9]:
parsed_input_rdd.take(5)

[Row(dtg=datetime.datetime(2003, 4, 1, 0, 10), location='260_T_a', name='De Bilt', latitude=52.098889, longitude=5.179722, altitude=1.9, u_bool_10=1.0, t_dryb_10=0.8, tn_10cm_past_6h_10=-4.8, t_dewp_10=0.4, t_dewp_sea_10=None, t_dryb_sea_10=None, tn_dryb_10=0.4, t_wetb_10=0.684949, tx_dryb_10=1.2, u_10=98.0, u_sea_10=None),
 Row(dtg=datetime.datetime(2003, 4, 1, 0, 20), location='260_T_a', name='De Bilt', latitude=52.098889, longitude=5.179722, altitude=1.9, u_bool_10=1.0, t_dryb_10=0.8, tn_10cm_past_6h_10=-4.8, t_dewp_10=0.5, t_dewp_sea_10=None, t_dryb_sea_10=None, tn_dryb_10=0.7, t_wetb_10=0.684949, tx_dryb_10=0.8, u_10=98.0, u_sea_10=None),
 Row(dtg=datetime.datetime(2003, 4, 1, 0, 30), location='260_T_a', name='De Bilt', latitude=52.098889, longitude=5.179722, altitude=1.9, u_bool_10=1.0, t_dryb_10=0.9, tn_10cm_past_6h_10=-4.8, t_dewp_10=0.6, t_dewp_sea_10=None, t_dryb_sea_10=None, tn_dryb_10=0.6, t_wetb_10=0.784415, tx_dryb_10=0.9, u_10=98.0, u_sea_10=None),
 Row(dtg=datetime.date

For calculating heat and cold waves, we only care about the fields:
1. datetime (DTG)
2. temperature (T_DRYB_10)
3. minimum temperature (TN_DRYB_10)
4. maximum temperature (TX_DRYB_10)

We can disregard all other fields for further exploration.

In [10]:
parsed_input_rdd = (
    parsed_input_rdd
    .map(lambda row: Row(dt=row.dtg, temperature=row.t_dryb_10, min_temperature=row.tn_dryb_10, max_temperature=row.tx_dryb_10))
)

In [11]:
parsed_input_rdd.take(5)

[Row(dt=datetime.datetime(2003, 4, 1, 0, 10), temperature=0.8, min_temperature=0.4, max_temperature=1.2),
 Row(dt=datetime.datetime(2003, 4, 1, 0, 20), temperature=0.8, min_temperature=0.7, max_temperature=0.8),
 Row(dt=datetime.datetime(2003, 4, 1, 0, 30), temperature=0.9, min_temperature=0.6, max_temperature=0.9),
 Row(dt=datetime.datetime(2003, 4, 1, 0, 40), temperature=0.6, min_temperature=0.6, max_temperature=0.9),
 Row(dt=datetime.datetime(2003, 4, 1, 0, 50), temperature=0.6, min_temperature=0.5, max_temperature=0.8)]

To make our live a bit easier for a further dive into the data, we create a DataFrame from the RDD to access functions at a higher level of abstraction.

In [12]:
schema = (
    StructType()
    .add("dt", TimestampType())
    .add("temperature", DoubleType())
    .add("min_temperature", DoubleType())
    .add("max_temperature", DoubleType())
)

df = (
    spark
    .createDataFrame(parsed_input_rdd, schema=schema)
)

In [13]:
df.show(5)

+-------------------+-----------+---------------+---------------+
|                 dt|temperature|min_temperature|max_temperature|
+-------------------+-----------+---------------+---------------+
|2003-04-01 00:10:00|        0.8|            0.4|            1.2|
|2003-04-01 00:20:00|        0.8|            0.7|            0.8|
|2003-04-01 00:30:00|        0.9|            0.6|            0.9|
|2003-04-01 00:40:00|        0.6|            0.6|            0.9|
|2003-04-01 00:50:00|        0.6|            0.5|            0.8|
+-------------------+-----------+---------------+---------------+
only showing top 5 rows



We aggregate the data to identify missing values, and identify potential outliers.

In [14]:
stats_df = (
    df
    .groupBy(
        f.year("dt").alias("year")
    )
    .agg(
        f.count("*").alias("num_rows"),
        f.count("dt").alias("num_dates"),
        f.count("temperature").alias("num_temps"),
        f.min("temperature").alias("min_temp"),
        f.max("temperature").alias("max_temp"),
        f.count("min_temperature").alias("num_min_temps"),
        f.min("min_temperature").alias("min_min_temp"),
        f.max("min_temperature").alias("max_min_temp"),
        f.count("max_temperature").alias("num_max_temps"),
        f.min("max_temperature").alias("min_max_temp"),
        f.max("max_temperature").alias("max_max_temp"),
    )
    .orderBy("year")
)

In [15]:
stats_df.show()

+----+--------+---------+---------+--------+--------+-------------+------------+------------+-------------+------------+------------+
|year|num_rows|num_dates|num_temps|min_temp|max_temp|num_min_temps|min_min_temp|max_min_temp|num_max_temps|min_max_temp|max_max_temp|
+----+--------+---------+---------+--------+--------+-------------+------------+------------+-------------+------------+------------+
|2003|   39599|    39599|    39565|    -7.2|    34.6|        39560|        -7.3|        34.4|        39560|        -7.0|        35.0|
|2004|   52560|    52560|    52408|    -7.7|    32.3|        52399|        -7.8|        32.0|        52399|        -7.5|        32.5|
|2005|   52560|    52560|    52539|   -14.4|    32.7|        52530|       -14.4|        32.5|        52530|       -14.1|        32.8|
|2006|   52560|    52560|    52516|    -6.9|    35.5|        52513|        -7.3|        35.1|        52513|        -6.6|        35.7|
|2007|   52560|    52560|    52510|    -6.8|    31.4|        5

Observations:
* As we can see from the monthly input files, we don't have data for all month in the years 2003 and 2020.
* There should be 52560 records (one for each 10 minutes; 60 * 24 * 365 / 10) in each regular year and 52704 records in a leap year. For most years, we have complete data. Exceptions are 2010 with 52416 records, and 2015 with 51264 records. Given this, the data should be reasonably complete. As we will disregard missing data, there's no need to further worry about this.
* The window functions-based algorithm aggregates the records by day. As only max/min aggregations are used, we don't need to take care of duplicates.
* There are no unreasonable temperature outliers that would need further investigation.
* The minimum and maximum temperature columns' min and max values are close to the temperature columns min and max values across all years, giving no indication for a closer look.
* Compared to the temperature column (t_dryb_10), the minimum temperature(tn_dryb_10) and maximum temperature (tn_dryb_10) columns have a few  more missing readings (less than a dozen per year). To fill in those blanks, we can consider coalescing min/max temperature columns and the temperature column.